In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk

In [3]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


False

In [4]:
import re

In [5]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
# amazon=pd.read_csv("Eco_Friendly_Products_Test_Full.xlsx - Sheet1.csv")
# amazon=pd.read_csv("Amazon Reviews Validation Data_1.csv")
amazon=pd.read_csv("amazon_neg.csv")

In [7]:
amazon

,Reviews,Aspect
0,heating element has died,Durability
1,It is attractive,Aesthetics
2,It's attractive,Aesthetics
3,if the writing is left on for more than a few ...,Ease of Reprocessing
4,increasingly difficult to wipe off,Ease of Reprocessing
...,...,...
148,It doesn't have any little crevices or weird s...,Interference
149,AND I'm not using a paper cup for each cup of ...,Aesthetics
150,"Lastly, I have not had any leak issues",Performance
151,unfortunately one of the lids does not fit at all,Performance


In [8]:
# a=amazon["Review"]
a=amazon["Reviews"]

In [9]:
q=amazon["Aspect"]

In [10]:
all_values = q.str.split(',').explode()

In [11]:
unique_values = all_values.unique()

In [12]:
len_aspect=len(unique_values)

In [13]:
len_aspect

11

In [14]:
processed_series = a

In [15]:
processed_series

0                               heating element has died
1                                       It is attractive
2                                        It's attractive
3      if the writing is left on for more than a few ...
4                    increasingly difficult to wipe off 
                             ...                        
148    It doesn't have any little crevices or weird s...
149    AND I'm not using a paper cup for each cup of ...
150               Lastly, I have not had any leak issues
151    unfortunately one of the lids does not fit at all
152    cheap enough that if my husband loses one, we ...
Name: Reviews, Length: 153, dtype: object

In [16]:
def extract_nouns(words):
    """Extract and return nouns from a given list of words."""
    # Join the list of words into a sentence
    # sentence = ' '.join(words)

    # Tokenize and POS tag the words in the sentence
    tokens = nltk.word_tokenize(words)
    parts_of_speech = nltk.pos_tag(tokens)

    # Extract and return words tagged as NN, NNS, NNP, or NNPS
    return [word for word, pos in parts_of_speech if pos in ['NN', 'NNS', 'NNP', 'NNPS']]


In [17]:
nouns = processed_series.apply(extract_nouns)
print(nouns)

0                                              [element]
1                                                     []
2                                                     []
3                                        [writing, days]
4                                                     []
                             ...                        
148                      [crevices, weird, areas, stuff]
149    [paper, cup, cup, coffee, work, godless, colle...
150                                             [issues]
151                                               [lids]
152                                            [husband]
Name: Reviews, Length: 153, dtype: object


In [18]:
from textblob import TextBlob

def extract_nouns_textblob(sentence):
    
    """Extract and return nouns from a given sentence using TextBlob."""
    # sentence = ' '.join(sentence)
    blob = TextBlob(sentence)
    return [word for word, tag in blob.tags if tag in ["NN", "NNS", "NNP", "NNPS"]]


In [19]:
nouns_testblob = processed_series.apply(extract_nouns_textblob)
print(nouns_testblob)

0                                              [element]
1                                                     []
2                                                     []
3                                        [writing, days]
4                                                     []
                             ...                        
148                      [crevices, weird, areas, stuff]
149    [paper, cup, cup, coffee, work, godless, colle...
150                                             [issues]
151                                               [lids]
152                                            [husband]
Name: Reviews, Length: 153, dtype: object


In [20]:
import spacy

In [21]:
nlpt = spacy.load("en_core_web_sm")

In [22]:
def extract_nouns_spacy(sentence):
    """Extract and return nouns from a given sentence using spaCy."""
    # sentence = ' '.join(sentence)
    doc = nlpt(sentence)
    return [token.text for token in doc if token.pos_ in ["NOUN"]]

In [23]:
doc = processed_series.apply(extract_nouns_spacy)
doc

0                                     [heating, element]
1                                                     []
2                                                     []
3                                        [writing, days]
4                                                     []
                             ...                        
148                             [crevices, areas, stuff]
149    [paper, cup, cup, coffee, work, colleagues, pl...
150                                       [leak, issues]
151                                               [lids]
152                                            [husband]
Name: Reviews, Length: 153, dtype: object

In [24]:
# doc[1180]

In [25]:
def load_glove_model(glove_file):
    """Load the GloVe model from a file."""
    print("Loading GloVe Model")
    with open(glove_file, 'r') as file:
        model = {}
        for line in file:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            model[word] = embedding
        print("Done. {} words loaded!".format(len(model)))
    return model

glove_model = load_glove_model("/Users/divyamsobti/Downloads/glove.6B/glove.6B.100d.txt")


Loading GloVe Model
Done. 400000 words loaded!


# TFidf


In [26]:
def extract_nouns_tf(sentence):

    # Tokenize and POS tag the words in the sentence
    tokens = nltk.word_tokenize(sentence)
    parts_of_speech = nltk.pos_tag(tokens)
    x=[word for word, pos in parts_of_speech if pos in ['NN', 'NNS', 'NNP', 'NNPS']]
    x=" ".join(x)
    # Extract and return words tagged as NN, NNS, NNP, or NNPS
    return x

In [27]:
nouns_tf = processed_series.apply(extract_nouns_tf)

In [28]:
nouns_tf

0                                                element
1                                                       
2                                                       
3                                           writing days
4                                                       
                             ...                        
148                           crevices weird areas stuff
149    paper cup cup coffee work godless colleagues p...
150                                               issues
151                                                 lids
152                                              husband
Name: Reviews, Length: 153, dtype: object

In [29]:


def extract_nouns_textblob_tf(sentence):
    
    """Extract and return nouns from a given sentence using TextBlob."""
    blob = TextBlob(sentence)
    x=[word for word, tag in blob.tags if tag in ["NN", "NNS", "NNP", "NNPS"]]
    x=" ".join(x)
    return x


In [30]:
nouns_testblob_tf = processed_series.apply(extract_nouns_textblob_tf)
print(nouns_testblob_tf)

0                                                element
1                                                       
2                                                       
3                                           writing days
4                                                       
                             ...                        
148                           crevices weird areas stuff
149    paper cup cup coffee work godless colleagues p...
150                                               issues
151                                                 lids
152                                              husband
Name: Reviews, Length: 153, dtype: object


In [31]:
def extract_nouns_spacy_tf(sentence):
    """Extract and return nouns from a given sentence using spaCy."""
    doc = nlpt(sentence)
    x=[token.text for token in doc if token.pos_ in ["NOUN"]]
    x=" ".join(x)
    return x

In [32]:
doc_tf = processed_series.apply(extract_nouns_spacy_tf)
doc_tf

0                                  heating element
1                                                 
2                                                 
3                                     writing days
4                                                 
                          ...                     
148                           crevices areas stuff
149    paper cup cup coffee work colleagues planet
150                                    leak issues
151                                           lids
152                                        husband
Name: Reviews, Length: 153, dtype: object

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
# Step 2: Convert text to numerical vectors using TF-IDF
vectorizer = TfidfVectorizer()
nltk = vectorizer.fit_transform(nouns_tf).toarray()

In [35]:
spacy_=vectorizer.fit_transform(doc_tf).toarray()

In [36]:
TextBlob_=vectorizer.fit_transform(nouns_testblob_tf).toarray()

In [37]:
def sentence_vector(word_list, model, vector_size=100):
    """Generate a sentence vector by averaging the word vectors."""
    word_vectors = [model[word] for word in word_list if word in model]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)




In [38]:
sentences_testblob=np.array(nouns_testblob)
sentences_spacy=np.array(doc)
sentences_nltk=np.array(nouns)


In [39]:
sentence_vectors_testblob = np.array([sentence_vector(sentence, glove_model) for sentence in sentences_testblob])

In [40]:
sentence_vectors_spacy = np.array([sentence_vector(sentence, glove_model) for sentence in sentences_spacy])

In [41]:
sentence_vectors_nltk = np.array([sentence_vector(sentence, glove_model) for sentence in sentences_nltk])

# Cluster for 12 aspects

In [42]:
from sklearn.cluster import KMeans

num_clusters = len_aspect  # Example number of clusters

In [43]:
num_clusters = len_aspect  # Example number of clusters
kmeans_textblob = KMeans(n_clusters=num_clusters)
kmeans_textblob.fit(sentence_vectors_testblob)
labels_textblob = kmeans_textblob.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [44]:
kmeans_spacy = KMeans(n_clusters=num_clusters)
kmeans_spacy.fit(sentence_vectors_spacy)
labels_spacy = kmeans_spacy.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [45]:
kmeans_nltk = KMeans(n_clusters=num_clusters)
kmeans_nltk.fit(sentence_vectors_nltk)
labels_nltk = kmeans_nltk.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


# ploting

In [46]:
import plotly.express as px
import plotly.io as pio
# pio.renderers.default = "browser"

In [47]:
from sklearn.manifold import TSNE


In [48]:
import plotly.graph_objects as go
import plotly.io as pio



## textblobl

In [49]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [50]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [51]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [52]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [53]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [54]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [55]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [56]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [57]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# kmeans with 6 aspects

In [58]:
num_clusters = 6 # Example number of clusters

In [59]:
kmeans_textblob = KMeans(n_clusters=num_clusters)
kmeans_textblob.fit(sentence_vectors_testblob)
labels_textblob = kmeans_textblob.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [60]:
kmeans_spacy = KMeans(n_clusters=num_clusters)
kmeans_spacy.fit(sentence_vectors_spacy)
labels_spacy = kmeans_spacy.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [61]:
kmeans_nltk = KMeans(n_clusters=num_clusters)
kmeans_nltk.fit(sentence_vectors_nltk)
labels_nltk = kmeans_nltk.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



# ploting

## textblobl

In [62]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [63]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [64]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [65]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [66]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [67]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [68]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [69]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [70]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# GMM

In [71]:
from sklearn.mixture import GaussianMixture

In [72]:
gmm = GaussianMixture(n_components = 12)

In [73]:
gmm_6 = GaussianMixture(n_components = 6)

In [74]:
gmm.fit(sentence_vectors_nltk)
labels_nltk = gmm.predict(sentence_vectors_nltk)

In [75]:
gmm.fit(sentence_vectors_spacy)
labels_spacy = gmm.predict(sentence_vectors_spacy)

In [76]:
gmm.fit(sentence_vectors_testblob)
labels_textblob = gmm.predict(sentence_vectors_testblob)

# ploting

In [77]:
num_clusters=12

## textblobl

In [78]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [79]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [80]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [81]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [82]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [83]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [84]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [85]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [86]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


# 6 cluster

In [87]:
gmm_6.fit(sentence_vectors_nltk)
labels_nltk = gmm_6.predict(sentence_vectors_nltk)

In [88]:
gmm_6.fit(sentence_vectors_spacy)
labels_spacy = gmm_6.predict(sentence_vectors_spacy)

In [89]:
gmm_6.fit(sentence_vectors_testblob )
labels_textblob = gmm_6.predict(sentence_vectors_testblob)

In [90]:
num_clusters=6

## textblobl

In [91]:
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_testblob)

In [92]:


fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Textblob)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [93]:

# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_testblob) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_testblob)

# 3D Scatter plot
fig = go.Figure()

for i in range(num_clusters):
    indices = labels_textblob == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Textblob)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## spacy

In [94]:
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_spacy)

In [95]:
fig = go.Figure()


for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(Spacy)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)


In [96]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_spacy) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_spacy)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_spacy == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(Spacy)',
                  scene=dict(xaxis_title='t-SNE Dimension 1',
                             yaxis_title='t-SNE Dimension 2',
                             zaxis_title='t-SNE Dimension 3'))
fig.show()


## NLTK


In [97]:
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne = TSNE(n_components=2, perplexity=perplexity_value, random_state=0)
reduced_vectors_2d = tsne.fit_transform(sentence_vectors_nltk)

In [98]:
fig = go.Figure()
for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter(
        x=reduced_vectors_2d[indices, 0],
        y=reduced_vectors_2d[indices, 1],
        mode='markers',
        name=f'Cluster {i + 1}'
    ))

fig.update_layout(
    title='Sentence Clusters Visualized with t-SNE(NLTK)',
    xaxis_title='t-SNE Dimension 1',
    yaxis_title='t-SNE Dimension 2',
    legend_title="Clusters"
)

pio.show(fig)

In [99]:
# Perform t-SNE dimensionality reduction for 3D visualization
perplexity_value = min(30, len(sentence_vectors_nltk) - 1)
tsne_3d = TSNE(n_components=3,perplexity=perplexity_value, random_state=0)
reduced_vectors_3d = tsne_3d.fit_transform(sentence_vectors_nltk)

fig = go.Figure()

for i in range(num_clusters):
    indices = labels_nltk == i
    fig.add_trace(go.Scatter3d(
        x=reduced_vectors_3d[indices, 0],
        y=reduced_vectors_3d[indices, 1],
        z=reduced_vectors_3d[indices, 2],
        mode='markers',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(title='3D Sentence Clusters(NLTK)',
                    scene=dict(xaxis_title='t-SNE Dimension 1',
                                 yaxis_title='t-SNE Dimension 2',
                                 zaxis_title='t-SNE Dimension 3'))
fig.show()


In [100]:
# Create DataFrame
df_spacy = pd.DataFrame({
    'Sentence': doc,
    'Cluster': labels_spacy
})

# Show the DataFrame
print(df_spacy)

                                              Sentence  Cluster
0                                   [heating, element]        0
1                                                   []        1
2                                                   []        1
3                                      [writing, days]        4
4                                                   []        1
..                                                 ...      ...
148                           [crevices, areas, stuff]        0
149  [paper, cup, cup, coffee, work, colleagues, pl...        4
150                                     [leak, issues]        0
151                                             [lids]        3
152                                          [husband]        4

[153 rows x 2 columns]


In [101]:
# Create DataFrame
df_textblob = pd.DataFrame({
    'Sentence': sentences_testblob,
    'Cluster': labels_textblob
})

# Show the DataFrame
print(df_textblob)

                                              Sentence  Cluster
0                                            [element]        5
1                                                   []        1
2                                                   []        1
3                                      [writing, days]        2
4                                                   []        1
..                                                 ...      ...
148                    [crevices, weird, areas, stuff]        5
149  [paper, cup, cup, coffee, work, godless, colle...        5
150                                           [issues]        5
151                                             [lids]        4
152                                          [husband]        2

[153 rows x 2 columns]


In [102]:
# Create DataFrame
df_nltk = pd.DataFrame({
    'Sentence': sentences_nltk,
    'Cluster': labels_nltk
})

# Show the DataFrame
print(df_nltk)

                                              Sentence  Cluster
0                                            [element]        4
1                                                   []        1
2                                                   []        1
3                                      [writing, days]        3
4                                                   []        1
..                                                 ...      ...
148                    [crevices, weird, areas, stuff]        2
149  [paper, cup, cup, coffee, work, godless, colle...        2
150                                           [issues]        5
151                                             [lids]        1
152                                          [husband]        3

[153 rows x 2 columns]
